In [1]:
import Pkg
push!(LOAD_PATH, "../../");
Pkg.activate("mcmc_redistricting", shared=true)

using Revise
using RandomNumbers
using LiftedTreeWalk

  Activating project at `~/.julia/environments/mcmc_redistricting`


In [2]:
pctGraphPath = joinpath(".", "4x4.json")
precinct_name = "precinct"
county_name = "county"
population_col = "TOTPOP"
num_dists = 4
rng_seed = 454190
pop_dev = 0.0
cycle_walk_frac = 0.01
steps = 100000/cycle_walk_frac
edge_len= "length"
edge_weights= "connections"
boundary_perim = "boundary_perim"
area_col = "area"
iso_weight = 0.02

nodeData = Set([precinct_name, county_name, population_col, boundary_perim, 
                area_col])

base_graph = BaseGraph(pctGraphPath,  population_col,  
                inc_node_data=nodeData, edge_weights=edge_weights,
                area_col=area_col, node_border_col=boundary_perim,
                edge_perimeter_col=edge_len);

In [3]:
outercorners = Set([(0,1), (2,3), (3,7), (11,15), (14,15), (12,13), (8,12), (0,4)])
outermiddle = Set([(1,2), (7,11), (13,14), (4,8)])
innermiddle = Set([(5,6), (6,10), (9,10), (5,9)])

outercorners = Set([(0,1), (2,3), (3,7), (11,15), (14,15), (12,13), (8,12), (0,4)])
outermiddle = Set([(1,2), (7,11), (13,14), (4,8)])
innermiddle = Set([(5,6), (6,10), (9,10), (5,9)])

function modify_perims!(
    base_graph::BaseGraph, 
    col::String,
    weights::Vector{Float64}=Vector{Float64}(undef, 0),
    edge_sets::Vector{Set{Tuple{Int64, Int64}}}=
        Vector{Set{Tuple{Int64, Int64}}}(undef, 0);
    base_weight::Float64=1.0
)
    for edge in keys(base_graph.edge_attributes)    
        e1, e2 = collect(edge).-1
        etup = (min(e1, e2), max(e1, e2))
        found_huh = false
        for (es_ind, es) ∈ enumerate(edge_sets)
            if etup ∈ es
                base_graph.edge_attributes[edge][col] = weights[es_ind]
                found_huh = true
                break
            end
        end
        if !found_huh
            base_graph.edge_attributes[edge][col] = base_weight
        end
    end
end

modify_perims! (generic function with 3 methods)

In [4]:
edge_sets = [outercorners, outermiddle, innermiddle]
modify_perims!(base_graph, edge_len, [2.0,2.5,3.0], edge_sets)
graph = MultiLevelGraph(base_graph, [precinct_name]);

In [5]:
for (edge, d) in base_graph.edge_attributes
    @show edge, d[edge_len]
end

(edge, d[edge_len]) = (Set([6, 10]), 3.0)
(edge, d[edge_len]) = (Set([5, 9]), 2.5)
(edge, d[edge_len]) = (Set([5, 1]), 2.0)
(edge, d[edge_len]) = (Set([7, 3]), 1.0)
(edge, d[edge_len]) = (Set([13, 9]), 2.0)
(edge, d[edge_len]) = (Set([6, 2]), 1.0)
(edge, d[edge_len]) = (Set([2, 3]), 2.5)
(edge, d[edge_len]) = (Set([11, 12]), 1.0)
(edge, d[edge_len]) = (Set([5, 6]), 1.0)
(edge, d[edge_len]) = (Set([7, 11]), 3.0)
(edge, d[edge_len]) = (Set([16, 12]), 2.0)
(edge, d[edge_len]) = (Set([11, 10]), 3.0)
(edge, d[edge_len]) = (Set([2, 1]), 2.0)
(edge, d[edge_len]) = (Set([7, 8]), 1.0)
(edge, d[edge_len]) = (Set([10, 9]), 1.0)
(edge, d[edge_len]) = (Set([4, 3]), 2.0)
(edge, d[edge_len]) = (Set([6, 7]), 3.0)
(edge, d[edge_len]) = (Set([15, 14]), 2.5)
(edge, d[edge_len]) = (Set([15, 11]), 1.0)
(edge, d[edge_len]) = (Set([12, 8]), 2.5)
(edge, d[edge_len]) = (Set([10, 14]), 1.0)
(edge, d[edge_len]) = (Set([15, 16]), 2.0)
(edge, d[edge_len]) = (Set([13, 14]), 2.0)
(edge, d[edge_len]) = (Set([4, 8]), 

In [6]:
using Graphs, SimpleWeightedGraphs
""""""
function get_cut_edge_list(
    partition::LinkCutPartition;
    column::String="connections"
)
    graph = partition.graph
    num_dists = partition.num_dists
    raw_data = [zeros(Float64, num_dists) for _ = 1:num_dists]
    total = 0
    for e in edges(graph.simple_graph)
        n1, n2 = src(e), dst(e)
        d1, d2 = partition.node_to_dist[n1], partition.node_to_dist[n2]
        if d1 == d2
            continue
        end
        w = graph.edge_attributes[Set([n1, n2])][column]
        raw_data[d1][d2] += w
        raw_data[d2][d1] += w
    end
    data = Vector(undef, num_dists)
    for di = 1:num_dists
        ngb_counts = zeros(Float64, 5)
        ngb_data = raw_data[di]
        for ni = 1:num_dists
            ngb_counts[Int(ngb_data[ni])+1] += 1
        end
        data[di] = Tuple(ngb_counts)
    end
    
    return Set(data)
end

get_cut_edge_list

In [7]:
# grab number of possible linking edge choices for each plan in list

plans = [(1, 1, 2, 2, 1, 1, 2, 2, 3, 3, 4, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 3, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 4, 4, 1, 2, 4, 4),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 3, 1, 2, 2, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 3, 3, 1, 2, 2, 4, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 3, 3, 4, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 3, 4, 2, 3, 4, 4, 4),
(1, 2, 3, 3, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 4, 4),
(1, 2, 2, 2, 1, 3, 4, 2, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 3, 4, 1, 3, 4, 4),
(1, 2, 2, 2, 1, 3, 2, 4, 1, 3, 4, 4, 1, 3, 3, 4),
(1, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 4, 2, 3),
(1, 2, 2, 2, 1, 3, 3, 2, 1, 4, 3, 3, 1, 4, 4, 4),
(1, 1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 4, 2, 3, 1, 4, 2, 3, 3, 4, 4),
(1, 1, 2, 2, 3, 1, 1, 2, 3, 4, 4, 2, 3, 3, 4, 4),
(1, 2, 2, 3, 1, 2, 4, 3, 1, 2, 4, 3, 1, 4, 4, 3),
(1, 2, 2, 2, 1, 2, 3, 3, 1, 1, 4, 3, 4, 4, 4, 3),
(1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4),
(1, 2, 2, 2, 1, 1, 2, 3, 1, 4, 3, 3, 4, 4, 4, 3)]

orbitsize = [1, 8, 4, 2, 8, 8, 4, 4, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 2, 2, 2]
cut_edges_per_plan = [8, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 
                      12, 12, 12, 12, 12, 12, 12, 12]
sp_trees = [256, 16, 16, 16, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

prob_of_cut_edges_iso_g0 = Dict{Int64, Float64}()
prob_of_cut_edges_iso_g1 = Dict{Int64, Float64}()
prob_of_cut_edges_g1 = Dict{Int64, Float64}()

node_to_district = Dict{Tuple{Vararg{String}}, Int}()
partition_index = Dict()

for (pi, p) in enumerate(plans)
    p = plans[pi]
    for ii = 1:length(p)
        node_to_district[graph.id_to_partitions[1][ii]] = p[ii]
    end
    ml_partition = MultiLevelPartition(graph, node_to_district)
    rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
    partition = LinkCutPartition(ml_partition, rng);
    isoperimetric_scores = get_isoperimetric_scores(partition)
    cut_edges = get_cut_edge_sum(partition)
    lst = get_cut_edge_list(partition)
    sfs = Int(round(exp(get_log_spanning_forests(partition))))
    orbitsz = orbitsize[pi]
    prob_of_cut_edges_iso_g0[cut_edges] = get(prob_of_cut_edges_iso_g0, cut_edges, 0) + orbitsz*sfs*exp(-iso_weight*sum(isoperimetric_scores))
    prob_of_cut_edges_iso_g1[cut_edges] = get(prob_of_cut_edges_iso_g1, cut_edges, 0) + orbitsz*exp(-iso_weight*sum(isoperimetric_scores))
    prob_of_cut_edges_g1[cut_edges] = get(prob_of_cut_edges_g1, cut_edges, 0) + orbitsz
    @show pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst
    k = (cut_edges, get_cut_edge_list(partition))
    if haskey(partition_index, k)
        partition_index[k] = Tuple([collect(partition_index[k]); pi])
    else
        partition_index[k] = (pi,)
    end
end

niso0inv = 1.0/sum(values(prob_of_cut_edges_iso_g0)) # need to normalize
niso1inv = 1.0/sum(values(prob_of_cut_edges_iso_g1)) # need to normalize
n1inv = 1.0/sum(values(prob_of_cut_edges_g1)) # need to normalize
for ce in keys(prob_of_cut_edges_iso_g1)
    prob_of_cut_edges_iso_g0[ce] *= niso0inv
    prob_of_cut_edges_iso_g1[ce] *= niso1inv
    prob_of_cut_edges_g1[ce] *= n1inv
end

@show prob_of_cut_edges_iso_g0
@show prob_of_cut_edges_iso_g1
@show prob_of_cut_edges_g1

(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (1, 8, 225.0, 1, Set(Any[(2.0, 0.0, 2.0, 0.0, 0.0)]))
(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (2, 10, 219.625, 8, Set(Any[(1.0, 1.0, 1.0, 0.0, 1.0), (1.0, 2.0, 1.0, 0.0, 0.0), (2.0, 0.0, 2.0, 0.0, 0.0), (2.0, 1.0, 0.0, 0.0, 1.0)]))
(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (3, 10, 238.75, 4, Set(Any[(1.0, 0.0, 2.0, 0.0, 1.0), (2.0, 0.0, 2.0, 0.0, 0.0), (3.0, 0.0, 0.0, 0.0, 1.0)]))
(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (4, 10, 170.0, 2, Set(Any[(1.0, 0.0, 3.0, 0.0, 0.0), (2.0, 0.0, 2.0, 0.0, 0.0)]))
(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (5, 11, 213.0, 8, Set(Any[(2.0, 0.0, 2.0, 0.0, 0.0), (1.0, 1.0, 1.0, 1.0, 0.0), (2.0, 0.0, 0.0, 2.0, 0.0)]))
(pi, cut_edges, sum(isoperimetric_scores), orbitsz, lst) = (6, 11, 199.375, 8, Set(Any[(2.0, 1.0, 0.0, 1.0, 0.0), (1.0, 2.0, 0.0, 1.0, 0.0), (1.0, 0.0, 0.0, 3.0, 0.0)]))
(pi, cut_edges, sum(isoperimetric_scores),

Dict{Int64, Float64} with 4 entries:
  11 => 0.205128
  10 => 0.119658
  12 => 0.666667
  8  => 0.00854701

In [29]:
# uniform measure on forests

constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 4052159124)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]

measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 

newsteps = steps/10
            
part_key_counts = Dict{Tuple, Int64}()
edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:newsteps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    lst = get_cut_edge_list(partition)
    pi = partition_index[(cut_edges, lst)]
    part_key_counts[pi] = get(part_key_counts, pi, 0) + 1
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

println("run 1")
for (k, v) in part_key_counts
    tot_p = 0
    for ki in k
        tot_p += orbitsize[ki]/117
    end
    @show k, v, v/steps, tot_p
end

println()
ce_to_count = Dict(8=>1,10=>14,11=>24,12=>78)
for (k, v) in edge_cut_counts
    tot_p = ce_to_count[k]/117
    @show k, v, v/steps, tot_p
end

run 1
(k, v, v / steps, tot_p) = ((13,), 142161, 0.0710805, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((15,), 146706, 0.073353, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((21,), 35703, 0.0178515, 0.017094017094017096)
(k, v, v / steps, tot_p) = ((2,), 135198, 0.067599, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((10,), 152811, 0.0764055, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((18,), 68538, 0.034269, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((4,), 37069, 0.0185345, 0.017094017094017096)
(k, v, v / steps, tot_p) = ((16,), 65364, 0.032682, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((12,), 141068, 0.070534, 0.06837606837606838)
(k, v, v / steps, tot_p) = ((8,), 64627, 0.0323135, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((17,), 61743, 0.0308715, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((1,), 16305, 0.0081525, 0.008547008547008548)
(k, v, v / steps, tot_p) = ((19,), 68494, 0.034247, 0.03418803418803419)
(k, v, v / steps, tot_p) = ((6,), 

In [14]:
# uniform measure
constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591241231)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]
measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 

newsteps = steps/10
edge_cut_counts = Dict{Int64, Int64}()
for ii = 1:newsteps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

for (k, v) in edge_cut_counts
    f = v/newsteps
    f_ex = prob_of_cut_edges_g1[k] 
    err = f_ex-f
    @show k, v, f, f_ex, err
end

(k, v, f, f_ex, err) = (11, 206099, 0.206099, 0.20512820512820515, -0.0009707948717948556)
(k, v, f, f_ex, err) = (10, 109737, 0.109737, 0.11965811965811968, 0.009921119658119676)
(k, v, f, f_ex, err) = (12, 676741, 0.676741, 0.6666666666666667, -0.010074333333333296)
(k, v, f, f_ex, err) = (8, 7423, 0.007423, 0.008547008547008548, 0.001124008547008548)


In [15]:
# uniform measure on forests weighted by polsby popper
constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591241231)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]
measure = Measure()
# push_energy!(measure, get_log_spanning_forests, 1.0) 
push_energy!(measure, get_isoperimetric_score, iso_weight) 

newsteps = steps/5
# newsteps = 1000
edge_cut_counts = Dict{Int64, Int64}()
pind_to_vals = Dict()
isoperimetric_score = get_isoperimetric_score(partition)
@show isoperimetric_score
for ii = 1:newsteps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng, pind_to_vals);
    cut_edges = get_cut_edge_sum(partition)
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

for (k, v) in edge_cut_counts
    f = v/newsteps
    f_ex = prob_of_cut_edges_iso_g0[k] 
    err = f_ex-f
    @show k, v, f, f_ex, err
end

isoperimetric_score = 225.0
(k, v, f, f_ex, err) = (11, 539787, 0.2698935, 0.25841684734768827, -0.011476652652311736)
(k, v, f, f_ex, err) = (10, 687333, 0.3436665, 0.3356823967625656, -0.00798410323743437)
(k, v, f, f_ex, err) = (12, 212772, 0.106386, 0.10676103264645417, 0.0003750326464541792)
(k, v, f, f_ex, err) = (8, 560108, 0.280054, 0.29913972324329197, 0.01908572324329194)


In [17]:
# uniform measure on partitions weighted by polsby popper
constraints = initialize_constraints()
add_constraint!(constraints, PopulationConstraint(graph, num_dists, pop_dev))

rng = PCG.PCGStateOneseq(UInt64, rng_seed + 40521591241231)
initial_partition = MultiLevelPartition(graph, constraints, num_dists; rng=rng);
partition = LinkCutPartition(initial_partition, rng);

cycle_walk = build_lifted_tree_cycle_walk(constraints)
internal_walk = build_internal_forest_walk(constraints)
proposal = [(cycle_walk_frac, cycle_walk), 
            (1.0-cycle_walk_frac, internal_walk)]
measure = Measure()
push_energy!(measure, get_log_spanning_forests, 1.0) 
push_energy!(measure, get_isoperimetric_score, iso_weight) 

newsteps = steps/10
# newsteps = 1000
edge_cut_counts = Dict{Int64, Int64}()
isoperimetric_score = get_isoperimetric_score(partition)
@show isoperimetric_score
for ii = 1:newsteps
    run_metropolis_hastings!(partition, proposal, measure, 1, rng);
    cut_edges = get_cut_edge_sum(partition)
    edge_cut_counts[cut_edges] = get(edge_cut_counts, cut_edges, 0) + 1
end

for (k, v) in edge_cut_counts
    f = v/newsteps
    f_ex = prob_of_cut_edges_iso_g1[k] 
    err = f_ex-f
    @show k, v, f, f_ex, err
end

isoperimetric_score = 225.0
(k, v, f, f_ex, err) = (11, 330660, 0.33066, 0.3338478987335233, 0.0031878987335232645)
(k, v, f, f_ex, err) = (10, 116713, 0.116713, 0.10841675373648768, -0.008296246263512316)
(k, v, f, f_ex, err) = (12, 546511, 0.546511, 0.5516969467193471, 0.005185946719347134)
(k, v, f, f_ex, err) = (8, 6116, 0.006116, 0.006038400810641972, -7.759918935802777e-5)
